In [56]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mlos_core.optimizers
import ConfigSpace as CS
from ConfigSpace import UniformIntegerHyperparameter
from ConfigSpace import UniformFloatHyperparameter
from ConfigSpace import CategoricalHyperparameter

In [57]:
def cost_model(index):
    performance_data = pd.read_csv('../../build/performance/performance_metrics.csv', header=None, names=["empty_reads", "non_empty_reads", "range_reads", "writes"])
    return performance_data.iloc[index:].mean().sum()

In [58]:
# Start defining a ConfigurationSpace for the Optimizer to search.
input_space = CS.ConfigurationSpace(seed=1234)

max_background_jobs=CS.Integer("max_background_jobs", (1,20), default=2)
max_subcompactions=CS.Integer("max_subcompactions", (1,20), default=1)
max_write_buffer_number=CS.Integer("max_write_buffer_number", (1,20), default=2)

max_bytes_for_level_multiplier=CS.Float("max_bytes_for_level_multiplier", (2,20.0), default=10)
level0_slowdown_writes_trigger=CS.Integer("level0_slowdown_writes_trigger", (1,50), default=20) 
# level0_stop_writes_trigger=CS.Integer("level0_stop_writes_trigger", (1,50), default=36) 

target_file_size_base=CS.Integer("target_file_size_base", (1,500 * 1048576), default=64 * 1048576)
target_file_size_multiplier=CS.Integer("target_file_size_multiplier", (1,20), default=1)

write_buffer_size=CS.Integer("write_buffer_size",(1,1024<<20),default=64<<20)
avoid_flush_during_shutdown=CS.Categorical("avoid_flush_during_shutdown", [True,False], default=True) 
IncreaseParallelism=CS.Integer("IncreaseParallelism",(1,16),default=1)

max_bytes_for_level_base=CS.Integer("max_bytes_for_level_base",(1,1000 * 1048576),default=256 * 1048576)
level0_file_num_compaction_trigger=CS.Integer("level0_file_num_compaction_trigger",(0,200),default=4)

In [61]:
input_space.add_hyperparameters([max_background_jobs,
                                 max_subcompactions,
                                 max_write_buffer_number,
                                 max_bytes_for_level_multiplier,
                                 level0_slowdown_writes_trigger,
#                                  level0_stop_writes_trigger])
                                target_file_size_base,
                                 target_file_size_multiplier,
                                 avoid_flush_during_shutdown,
                                IncreaseParallelism,
                                write_buffer_size,
                                max_bytes_for_level_base,
                                level0_file_num_compaction_trigger])

[max_background_jobs, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_subcompactions, Type: UniformInteger, Range: [1, 20], Default: 1,
 max_write_buffer_number, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_bytes_for_level_multiplier, Type: UniformFloat, Range: [2.0, 20.0], Default: 10.0,
 level0_slowdown_writes_trigger, Type: UniformInteger, Range: [1, 50], Default: 20,
 target_file_size_base, Type: UniformInteger, Range: [1, 524288000], Default: 67108864,
 target_file_size_multiplier, Type: UniformInteger, Range: [1, 20], Default: 1,
 avoid_flush_during_shutdown, Type: Categorical, Choices: {True, False}, Default: True,
 IncreaseParallelism, Type: UniformInteger, Range: [1, 16], Default: 1,
 write_buffer_size, Type: UniformInteger, Range: [1, 1073741824], Default: 67108864,
 max_bytes_for_level_base, Type: UniformInteger, Range: [1, 1048576000], Default: 268435456,
 level0_file_num_compaction_trigger, Type: UniformInteger, Range: [0, 200], Default: 4]

In [62]:
#optimizer = mlos_core.optimizers.RandomOptimizer(parameter_space=input_space)

#optimizer = mlos_core.optimizers.FlamlOptimizer(parameter_space=input_space)

optimizer = mlos_core.optimizers.SmacOptimizer(parameter_space=input_space) # , seed=42, n_random_init=20)

In [63]:
optimizer

SmacOptimizer(space_adapter=None)

In [64]:
import os
import time
import sys
def pass_values_to_interface(new_val):
    #print(new_val)
    pipe_path = '../../build/passing_params_pipe'
    if not os.path.exists(pipe_path):
        os.mkfifo(pipe_path)
    pipe_fd = os.open(pipe_path, os.O_RDWR)
    data =''
    try:
        for x in new_val:
            data+='\n' + x + '=' + str(new_val[x].iloc[0])
        data=data.encode('utf-8')
        os.write(pipe_fd, data)
        sys.stdout.flush()
    except KeyboardInterrupt:
        pass
#     finally:
#         os.close(pipe_fd)


In [65]:
def read_epochs_pipe():
    while True:
        epochs_pipe_path = '../../build/passing_epochs'
        if os.path.exists(epochs_pipe_path):
            epochs_pipe_fd = os.open(epochs_pipe_path, os.O_RDONLY)
            try:
                buffer_size = 128
                buffer = os.read(epochs_pipe_fd, buffer_size).decode('utf-8')
                if buffer:
                    reported_epochs = int(buffer)
                    print(f"Reported epochs: {reported_epochs}")
                    return reported_epochs
            except OSError as e:
                if e.errno == os.errno.EAGAIN or e.errno == os.errno.EWOULDBLOCK:
                    pass
                else:
                    raise
            finally:
                os.close(epochs_pipe_fd)
        else:
            pass
    


In [66]:
def run_optimization():
    # get a new config value suggestion to try from the optimizer.
    suggested_value = optimizer.suggest()
    pass_values_to_interface(suggested_value)
    index=read_epochs_pipe()
    time.sleep(5)
    if index==-1:
        target_value=float('inf')
    else:
        target_value = cost_model(index)
    optimizer.register(suggested_value, pd.Series([target_value]))


n_iterations = 50
for i in range(n_iterations):
    print("epoch:",i)
    run_optimization()

epoch: 0
Reported epochs: 544
epoch: 1
Reported epochs: 656
epoch: 2
Reported epochs: 754
epoch: 3
Reported epochs: 844
epoch: 4
Reported epochs: 927
epoch: 5
Reported epochs: 1006
epoch: 6
Reported epochs: 1080
epoch: 7
Reported epochs: 1150
epoch: 8
Reported epochs: 12161278
epoch: 9
Reported epochs: 133713961452
epoch: 10
Reported epochs: 150615591610
epoch: 11
Reported epochs: 165917081755
epoch: 12
Reported epochs: 180018441888
epoch: 13
Reported epochs: 193119732014
epoch: 14
Reported epochs: 205320872118
epoch: 15
Reported epochs: 214421812216
epoch: 16
Reported epochs: 224322792314
epoch: 17
Reported epochs: 234823772400
epoch: 18
Reported epochs: 243124632492
epoch: 19
Reported epochs: 252525562584
epoch: 20
Reported epochs: 26172648
epoch: 21
Reported epochs: 267927102741
epoch: 22
Reported epochs: 277228022832
epoch: 23
Reported epochs: 286228922921
epoch: 24
Reported epochs: 295029793008
epoch: 25
Reported epochs: 303730653093
epoch: 26
Reported epochs: 312131493176
epoch: 

In [13]:
optimizer.get_observations()

,max_bytes_for_level_multiplier,score
0,12.705253,32.633333
0,2.407490,35.692308
0,7.437141,36.043478
0,16.686254,36.545455
0,19.140087,38.105263
0,9.755837,35.777778
0,4.729418,36.000000
0,15.173295,38.437500
0,13.543099,37.875000
0,6.051800,39.200000


In [14]:
optimizer.get_best_observation()

,max_bytes_for_level_multiplier,score
0,12.705253,32.633333


In [15]:
optimum_value=optimizer.suggest()
pass_values_to_interface(optimum_value)

In [16]:
print(optimum_value)

   max_bytes_for_level_multiplier
0                        2.192011


In [ ]:
hyper_parameters_integer = {
    "max_open_files":[-1,10000, -1],
    "max_total_wal_size":[0, 100000, 0],
    "delete_obsolete_files_period_micros":[0, 6 * 60 * 60 * 1000000, 6 * 60 * 60 * 1000000],
    "max_background_jobs":[2, 10, 2],
    "max_subcompactions":[1, 100, 1],
    "compaction_readahead_size":[0, 1000000, 0],
    "writable_file_max_buffer_size":[0, 1024 * 1024, 1024 * 1024],
    "delayed_write_rate":[0, 1000000, 0],
    
    "max_write_buffer_number":[1, 10, 2],
    "inplace_update_num_locks":[0, 10000, 0],
    "memtable_huge_page_size":[0, 1000000, 0],
    "arena_block_size":[0, 1000000, 0],
    "level0_slowdown_writes_trigger":[0, 100, 20],
    "level0_stop_writes_trigger":[0, 100, 36],
    "target_file_size_base":[0, 100000000, 64 * 1048576],
    "target_file_size_multiplier":[0, 100, 1],
    "max_compaction_bytes":[0, 1000000000, 64 * 1048576 * 25],
    "soft_pending_compaction_bytes_limit":[0, 1000000000, 64 * 1073741824],
    "hard_pending_compaction_bytes_limit":[0, 1000000000, 256 * 1073741824]                          
}
                                           
hyper_parameters_float ={
    "memtable_prefix_bloom_size_ratio":[0.0, 1.0, 0.0],
    "max_bytes_for_level_multiplier":[0.0, 100.0, 10.0]
}
                                           
hyper_parameters_catagorical ={
    "avoid_flush_during_shutdown":[[True, False],False],  
    "memtable_whole_key_filtering":[[True, False],False],
   "compression_per_level":[["kNoCompression", "kSnappyCompression", "kZlibCompression", "kBZip2Compression", "kLZ4Compression", "kLZ4HCCompression", "kXpressCompression", "kZSTD"], "kNoCompression"]

}